In [ ]:
from bs4 import BeautifulSoup
import requests
from retrying import retry

## 防止访问失败的重试操作

In [ ]:
@retry(stop_max_attempt_number=100) #最大重试100次，100次全部报错，才会报错
def _open_url(url):
    response = requests.get(url, headers={
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'
    }, timeout=10) #超时的时候回报错并重试
    assert response.status_code == 200 #状态码不是200，也会报错并充实
    return response

## 得到页面上符合条件的链接

In [ ]:
def get_menu(url):#得到页面上的超链接目录
    response = _open_url(url)
    response.encoding = 'utf-8'
    data = response.text
    soup = BeautifulSoup(data,features="lxml")
    urls=[]
    for item in soup.find_all("a"):
        if item.string == None:
            continue
        else:
            urls.append('https://www.xxxx.net'+item.get("href"))
    
    urls_real=[]
    for line in urls:
        if url in line:
            urls_real.append(line)
        
#     for i in range(len(urls_real)):
#         print(str(urls_real[i]))
    return urls_real

## 无关文字过滤器，顺便实现了一些排版功能

In [ ]:
def textfilter(text,title):# 注意顺序
    text = text.replace("天才一秒记住本站地址：[笔趣阁]","")
    text = text.replace("https://www.ibiquge.net/最快更新！无广告！","")
    text = text.replace("章节错误,点此报送(免注册),","")
    text = text.replace("报送后维护人员会在两分钟内校正章节内容,请耐心等待。","")
    text = text.replace("\xa0\xa0\xa0\xa0","\n")
    text = text.replace("\n\r\n                \n\r\n                \n\r\n            \n","")
    text = text.replace("           \r\n            \n\r\n                \r\n            \n","")
    text = text.replace("\xa0"," ")
    text = text.replace(title,"")# 正文里面有标题，让我们很生气地删掉！
    return text

## 得到章节的标题和内容

In [ ]:
def get_text(url): #得到页面上的目录
    response = _open_url(url)
    response.encoding = 'utf-8'
    data = response.text
    soup = BeautifulSoup(data,features="lxml")
    title = soup.find('h1').get_text()
    text = soup.find(id='content').get_text()
    text = textfilter(text,title)# 过滤无关文字
    return [title,text]

## 开始读取内容，并且存储

In [ ]:
url = 'https://www.xxxx.net/bookid/'
menu = get_menu(url)
menu = menu[13:] # 去掉前面章节

In [ ]:
path = r'./'
bookname='斗不破苍穹'
for url in menu:
    [title,text] = get_text(url)
    with open('./%s.txt' % bookname,'a+',encoding='utf-8') as f:
        f.write(title+'\n'+text+'\n\n')
    print("%s  储存完成！" % title)